# Getting Pubchem FPs for non-downsampled data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import scipy
from scipy import stats
from scipy.stats import chisquare

#hiding warning messages
import warnings
warnings.filterwarnings("ignore")
import time 

import urllib
from urllib.request import urlopen
from bs4 import BeautifulSoup

#Reading in Molecular Properties CSV
data = pd.read_csv('Molecular_Properties_CSV')
act_list = data['Activity Outcome'].tolist()
CID_list = data['PUBCHEM_CID'].tolist()
#data = data.astype(float, errors = 'ignore')
data.head()

,PUBCHEM_CID,Activity Outcome,MolecularFormula,MolecularWeight,HBondDonorCount,HBondAcceptorCount,RotatableBondCount,CanonicalSMILES,IsomericSMILES,InChI,XLogP,ExactMass,TPSA,HeavyAtomCount,Complexity
0,4,inactive,C3H9NO,75.11,2,2,1,CC(CN)O,CC(CN)O,"InChI=1S/C3H9NO/c1-3(5)2-4/h3,5H,2,4H2,1H3",-1.0,75.068414,46.2,5,22.9
1,11,inactive,C2H4Cl2,98.96,0,0,1,C(CCl)Cl,C(CCl)Cl,InChI=1S/C2H4Cl2/c3-1-2-4/h1-2H2,1.5,97.969005,0.0,4,6.0
2,13,inactive,C6H3Cl3,181.40,0,0,0,C1=CC(=C(C=C1Cl)Cl)Cl,C1=CC(=C(C=C1Cl)Cl)Cl,InChI=1S/C6H3Cl3/c7-4-1-2-5(8)6(9)3-4/h1-3H,4.0,179.930033,0.0,9,94.3
3,33,inactive,C2H3ClO,78.50,0,1,1,C(C=O)Cl,C(C=O)Cl,"InChI=1S/C2H3ClO/c3-1-2-4/h2H,1H2",0.3,77.987242,17.1,4,20.0
4,34,inactive,C2H5ClO,80.51,1,1,1,C(CCl)O,C(CCl)O,"InChI=1S/C2H5ClO/c3-1-2-4/h4H,1-2H2",-0.1,80.002893,20.2,4,10.0


In [2]:
target_list = []
i = 0

while (i < len(act_list)):
    if (act_list[i] == 'active antagonist'):
        target_list.append(1)
        i = i + 1
    elif (act_list[i] == 'active agonist'):
        target_list.append(1)
        i = i + 1
    else:
        target_list.append(0)
        i = i + 1
        
#Making strings ints
target_list = [int(i) for i in target_list]

#adding target column to data
data['target'] = target_list

In [3]:
#Getting list of pubchem fp values for all CID
#takes like an hr to run

pub_fp = []
i = 0
while i < len(CID_list):
    try:
    # (list_cov_multi[i] == '2'):
        url1 = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/' + str(CID_list[i]) + '/property/Fingerprint2D/TXT'
        html1 = urlopen(url1) 
        soup1 = BeautifulSoup(html1, 'lxml')
        fp1 = soup1.get_text()
        pub_fp.append(fp1)
        i = i + 1
       # else:
        #    i = i + 1
    except:
        time.sleep(1)
        i = i
        #XLogP.append('None')
        #i = i + 1

len(pub_fp)

5672

In [4]:
#Removing '\n from the end of each XLogP value in list: XLogP'
bad_chars = ['\n']

i = 0
while i < len(pub_fp):
    if pub_fp[i] == 'None':
        i = i + 1
    else:
        for j in bad_chars: 
            pub_fp[i] = pub_fp[i].replace(j, '')
            i = i + 1


In [5]:
#Decoding Fingerprints
from base64 import b64decode

def PCFP_BitString(pcfp_base64) :

    pcfp_bitstring = "".join( ["{:08b}".format(x) for x in b64decode( pcfp_base64 )] )[32:913]
    return pcfp_bitstring

i = 0
pub_fp_decoded = []
while (i < len(pub_fp)):
    fp = PCFP_BitString(pub_fp[i])
    pub_fp_decoded.append(fp)
    i = i + 1

In [6]:
#Adding activity_score/Making input train data
#making list of names etc.
i = 0
pub_fp_names = ['Name', 'Activity']
while (i < len(pub_fp_decoded[0])):
    string = "PubFP" + str(i + 1)
    pub_fp_names.append(string)
    i = i + 1

first_row = []
first_row.append(CID_list[0])
first_row.append(target_list[0])
first_row.extend([int(a) for a in str(pub_fp_decoded[0])])
full_pub = pd.DataFrame(first_row).T
x = 1
row = []
while (x < len(pub_fp_decoded)):
    bit_row1 = [int(y) for y in str(pub_fp_decoded[x])]
    row.append(CID_list[x])
    row.append(target_list[x])
    row.extend(bit_row1)
    row1df = pd.DataFrame(row).T
    full_pub = full_pub.append(row1df)
    row.clear()   
    x = x + 1

    
#Reindexing df, namesetc as column headers
full_pub.columns = [pub_fp_names]

In [7]:
full_pub.to_csv('full_pub_fp.csv', index = False)

In [8]:
full_pub.head()

,Name,Activity,PubFP1,PubFP2,PubFP3,PubFP4,PubFP5,PubFP6,PubFP7,PubFP8,...,PubFP872,PubFP873,PubFP874,PubFP875,PubFP876,PubFP877,PubFP878,PubFP879,PubFP880,PubFP881
0,4,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,11,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,13,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,33,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,34,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Structure Alerts

In [9]:
#Reading in full pubchem fingerprint CSV
data = pd.read_csv('full_pub_fp.csv')
#data = data.astype(float, errors = 'ignore')
data.shape

(5672, 883)

In [10]:
#Split into actives and inactives
df_inactive = data.drop(data.loc[data['Activity'] == 1].index)
df_active = data.drop(data.loc[data['Activity'] == 0].index)

print(df_inactive.shape)
print(df_active.shape)

(5263, 883)
(409, 883)


In [11]:
fragment_list_inact = df_inactive.columns[2:].tolist()
fragment_list_act = df_active.columns[2:].tolist()

In [12]:
#For each fragment, getting the number of times it was present in an inactive molecule (1)
df_inactivating = pd.DataFrame()
for fragment in fragment_list_inact:
    value_counts_inact = df_inactive[fragment].value_counts()
    df_inactivating = df_inactivating.append(value_counts_inact)

#dropping NA values
#NA values indicate fragments that were not present at ALL in inactive compounds
df_inactivating = df_inactivating.dropna()

#renaming columns
df_inactivating = df_inactivating.rename(columns = {0: 'Not present in inactive', 1: 'Present in inactive' })
df_inactivating.shape

(691, 2)

In [13]:
#For each fragment, getting the number of times it was present in an active molecule (1)
df_activating = pd.DataFrame()
for fragment in fragment_list_act:
    value_counts_act = df_active[fragment].value_counts()
    df_activating = df_activating.append(value_counts_act)

#dropping NA values
#NA values indicate fragments that were not present at ALL in any active compounds
df_activating = df_activating.dropna()

#renaming columns
df_activating = df_activating.rename(columns = {0: 'Not present in active', 1: 'Present in active' })
df_activating.shape

(603, 2)

In [49]:
#combining df_inactivating/df_activating
df_all = pd.concat([df_inactivating, df_activating], axis = 1)

#drop the "not present in active/not present in inactive columns"
df_all = df_all.drop(columns = (['Not present in active', 'Not present in inactive']))

#filling na with 0
df_all = df_all.fillna(1)

#making copy 
df_copy = df_all

In [50]:
#find overall ratio between active/inactive compounds (1:1)
print(data['Activity'].value_counts())
print("Active/Inactive Ratio: " + str(409 / 5263))


0    5263
1     409
Name: Activity, dtype: int64
Active/Inactive Ratio: 0.0777123313699411


In [51]:
#make new column for sum of the actives/inactives, so the total number of times that fragment appears in the data
df_all['Sum'] = df_all['Present in active'] + df_all['Present in inactive']

#make new column for ratio of actives/inactives
df_all['Active/Inactive Ratio'] = df_all['Present in active'] / df_all['Present in inactive']


df_all['Expected'] = (409 / 5263)

In [52]:
#Generating chi squared value for each fp and adding as column to df
present_act = df_all['Present in active'].tolist()
present_inact = df_all['Present in inactive'].tolist()
act_inact_ratio = df_all['Active/Inactive Ratio'].tolist()
chisqr = []
i = 0
while (i < len(df_all.index)):
    if (present_act[i] == 0.0):
        chisqr.append(np.abs(present_act[i] - present_inact[i]))
        i = i + 1
    elif (present_inact[i] == 0.0):
        chisqr.append(np.abs(present_act[i] - present_inact[i]))
        i = i + 1
    else:
        statistic = ((act_inact_ratio[i] - (409/5263)) ** 2)/(409 / 5263)
        chisqr.append(statistic)
        i = i + 1
len(chisqr)

693

In [53]:
df_all['Chi Squared Statistic'] = chisqr

#Sorting by chi squared score
df_all = df_all.sort_values(by = ['Chi Squared Statistic'], ascending = False, inplace = False)

In [54]:
df_significant = df_all.loc[df_all['Chi Squared Statistic'] > 755]

In [61]:
mean_ratio = df_all['Active/Inactive Ratio'].mean()

In [66]:
sd = df_all['Active/Inactive Ratio'].std()

In [70]:
df_all['abs zscore'] = abs((df_all['Active/Inactive Ratio'] - mean_ratio)/sd)

In [73]:
df_significant = df_all.loc[df_all['abs zscore'] >= 1.96]
df_significant

,Present in inactive,Present in active,Sum,Active/Inactive Ratio,Expected,Chi Squared Statistic,zscore,abs zscore
PubFP867,1.0,14.0,15.0,14.000000,0.077712,2494.199962,22.063316,22.063316
PubFP841,31.0,112.0,143.0,3.612903,0.077712,160.818424,5.424367,5.424367
PubFP862,32.0,112.0,144.0,3.500000,0.077712,150.710353,5.243509,5.243509
PubFP96,1.0,3.0,4.0,3.000000,0.077712,109.889448,4.442566,4.442566
PubFP243,59.0,131.0,190.0,2.220339,0.077712,59.074910,3.193638,3.193638
PubFP81,1.0,2.0,3.0,2.000000,0.077712,47.549595,2.840680,2.840680
PubFP558,1.0,2.0,3.0,2.000000,0.077712,47.549595,2.840680,2.840680
PubFP124,1.0,2.0,3.0,2.000000,0.077712,47.549595,2.840680,2.840680
